In [27]:
import torch
import torchvision
import torchvision.transforms as transforms
from torch import nn, optim
from torchvision.datasets import MNIST
import torch.nn.functional as F
import wandb

In [28]:
torch.cuda.is_available()
torch.cuda.device(0)

In [29]:
wandb.init(project="mnist-model-evaluation", entity="kimkj")

wandb: WARNING Source type is set to 'repo' but some required information is missing from the environment. A job will not be created from this run. See https://docs.wandb.ai/guides/launch/create-job


In [30]:
transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.5,), (0.5,))])

trainset = MNIST(root='./data', train=True, download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=64, shuffle=True)

testset = MNIST(root='./data', train=False, download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=64, shuffle=False)


In [31]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, 5)  # input channel, output channels, kernel size
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(32, 64, 5)
        self.fc1 = nn.Linear(64 * 4 * 4, 128)
        self.fc2 = nn.Linear(128, 10)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = x.view(-1, 64 * 4 * 4)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x

In [32]:
net = Net()
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(net.parameters(), lr=0.001)

In [33]:
wandb.watch(net, log="all")


[]

In [34]:
for epoch in range(10):  # loop over the dataset multiple times
    running_loss = 0.0
    for i, data in enumerate(trainloader, 0):
        inputs, labels = data
        optimizer.zero_grad()
        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
        if i % 100 == 99:    # log every 100 mini-batches
            print(f'[{epoch + 1}, {i + 1}] loss: {running_loss / 100}')
            wandb.log({"loss": running_loss / 100})
            running_loss = 0.0

[1, 100] loss: 0.6118012794107198
[1, 200] loss: 0.17894582372158765
[1, 300] loss: 0.11059527533128857
[1, 400] loss: 0.09305913521908224
[1, 500] loss: 0.07619017046410591
[1, 600] loss: 0.07818639677017927
[1, 700] loss: 0.08075645588338375
[1, 800] loss: 0.06211619125446305
[1, 900] loss: 0.05760706997476518
[2, 100] loss: 0.04387284838361666
[2, 200] loss: 0.04518807751708664
[2, 300] loss: 0.047502816563937816
[2, 400] loss: 0.04860372927505523
[2, 500] loss: 0.046855244447942826
[2, 600] loss: 0.04483762062853202
[2, 700] loss: 0.04131639020284638
[2, 800] loss: 0.04579319661017507
[2, 900] loss: 0.03641025670105592
[3, 100] loss: 0.026309253356885165
[3, 200] loss: 0.028834886578843
[3, 300] loss: 0.030832766079111024
[3, 400] loss: 0.0294508678127022
[3, 500] loss: 0.02964957422052976
[3, 600] loss: 0.031584769579931164
[3, 700] loss: 0.033021427549829244
[3, 800] loss: 0.04542470301908907
[3, 900] loss: 0.024645655838539823
[4, 100] loss: 0.023206861291255335
[4, 200] loss: 0

In [35]:
correct = 0
total = 0
with torch.no_grad():
    for data in testloader:
        images, labels = data
        outputs = net(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print('Accuracy of the network on the 10000 test images: %d %%' % (100 * correct / total))
wandb.log({"accuracy": 100 * correct / total})

Accuracy of the network on the 10000 test images: 99 %


In [36]:
wandb.finish()


wandb: WARNING Source type is set to 'repo' but some required information is missing from the environment. A job will not be created from this run. See https://docs.wandb.ai/guides/launch/create-job


accuracy,▁
loss,█▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
accuracy,99.16
loss,0.00857
